In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.graph_objects as go


In [2]:


df=pd.read_csv(r"C:\Users\katch\Desktop\projects\stock_portfolio\notebooks\Data\processed_data\stock_price_data.csv")

df.head()



,Date,Ticker,Close,High,Low,Open,Volume
0,2020-06-30,ADANIGREEN.NS,358.850006,358.850006,358.850006,358.850006,126347.0
1,2020-06-30,BANKBEES.NS,215.410004,218.399994,213.800003,218.350006,969269.0
2,2020-06-30,BHEL.NS,35.301929,37.378512,34.609734,36.191892,81189594.0
3,2020-06-30,CPSEETF.NS,17.680000,18.180000,17.650000,18.000000,731372.0
4,2020-06-30,GOLDBEES.NS,42.610001,42.889999,42.419998,42.490002,4643078.0


In [3]:
df['Ticker'] = df['Ticker'].str.replace('.NS', '', regex=False)


In [4]:
df.head()

,Date,Ticker,Close,High,Low,Open,Volume
0,2020-06-30,ADANIGREEN,358.850006,358.850006,358.850006,358.850006,126347.0
1,2020-06-30,BANKBEES,215.410004,218.399994,213.800003,218.350006,969269.0
2,2020-06-30,BHEL,35.301929,37.378512,34.609734,36.191892,81189594.0
3,2020-06-30,CPSEETF,17.680000,18.180000,17.650000,18.000000,731372.0
4,2020-06-30,GOLDBEES,42.610001,42.889999,42.419998,42.490002,4643078.0


In [5]:
df['SMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
df['SMA_50'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=50).mean())
df['SMA_100'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=100).mean())
df['SMA_200'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=200).mean())

In [6]:
df['EMA_50'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=50, adjust=False).mean())
df['EMA_100'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=100, adjust=False).mean())
df['EMA_200'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=200, adjust=False).mean())

In [7]:
def plot(df, ticker, column_names):
    df = df[df['Ticker'] == ticker].copy()

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Open', 'High', 'Low', 'Close'] + column_names)

    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick'
    ))

    for col in column_names:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        fig.add_trace(go.Scatter(
            x=df['Date'],
            y=df[col],
            mode='lines',
            name=col,
            line=dict(width=2),
            fill=None,
            connectgaps=True
        ))

    fig.update_layout(
        title=f'{ticker} Candlestick + Indicators',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_dark',
        xaxis_rangeslider_visible=False,
        width=1200,
        height=700
    )

    fig.show()


In [8]:
plot(df,'ABB',['SMA_20','SMA_50','SMA_100','EMA_50','EMA_100'])

Bollinger bands

In [9]:
df['upper_band_bb']=df['SMA_20'] + (2 *df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).std()))
df['lower_band_bb']=df['SMA_20'] - (2 *df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).std()))

In [10]:
def plot_bollinger_bands(df,ticker):
    plot(df,ticker,['lower_band_bb','upper_band_bb','SMA_20'])


In [11]:
plot_bollinger_bands(df,'ABB')

In [12]:
df.tail()

,Date,Ticker,Close,High,Low,Open,Volume,SMA_20,SMA_50,SMA_100,SMA_200,EMA_50,EMA_100,EMA_200,upper_band_bb,lower_band_bb
12385,2025-06-27,HDFCSML250,177.399994,180.399994,176.089996,180.399994,1372077.0,173.293000,165.449000,157.424600,167.384150,166.853335,163.971905,163.186558,177.956555,168.629446
12386,2025-06-27,JUNIORBEES,733.929993,735.950012,730.700012,733.849976,448882.0,722.177499,709.619998,681.864200,717.384250,708.907148,702.166436,700.696713,739.286612,705.068387
12387,2025-06-27,MID150BEES,225.009995,227.000000,223.100006,227.000000,671525.0,221.507999,214.111000,203.652600,NaN,214.695341,210.325689,207.102500,226.711117,216.304880
12388,2025-06-27,NIFTYBEES,287.959991,288.279999,286.440002,287.000000,5139824.0,280.251500,276.828201,266.997601,269.426601,276.092326,271.641366,267.367759,286.511419,273.991581
12389,2025-06-27,^NSEI,25637.800781,25654.199219,25523.550781,25576.650391,564000.0,24977.715039,24710.352891,NaN,NaN,24630.026383,24252.677493,23907.790592,25540.621761,24414.808317


In [13]:
df.columns

Index(['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_20',
       'SMA_50', 'SMA_100', 'SMA_200', 'EMA_50', 'EMA_100', 'EMA_200',
       'upper_band_bb', 'lower_band_bb'],
      dtype='object')

pivot points

In [14]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values(['Ticker', 'Date'])

def calc_pivots(group):
    H = group['High']
    L = group['Low']
    C = group['Close']
    P = (H + L + C) / 3

    group['Pivot'] = P
    group['S1'] = (2 * P) - H
    group['S2'] = P - (H - L)
    group['S3'] = P - 2 * (H - L)
    group['R1'] = (2 * P) - L
    group['R2'] = P + (H - L)
    group['R3'] = P + 2 * (H - L)
    return group




In [15]:
df=df.groupby('Ticker').apply(calc_pivots)

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\2851656944.py:1: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [16]:
import plotly.graph_objects as go

def plot_pivots(df, ticker):
    # Ensure 'Ticker' is a column
    if 'Ticker' not in df.columns:
        df = df.reset_index()

    df_ticker = df[df['Ticker'] == ticker].sort_values('Date')

    if df_ticker.empty:
        print(f"No data for ticker: {ticker}")
        return

    last_row = df_ticker.iloc[-1]

    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(
        x=df_ticker['Date'],
        open=df_ticker['Open'],
        high=df_ticker['High'],
        low=df_ticker['Low'],
        close=df_ticker['Close'],
        name='Candlesticks'
    ))

    # Add horizontal pivot lines
    for level in ['Pivot', 'S1', 'S2', 'S3', 'R1', 'R2', 'R3']:
        y_val = last_row[level]
        fig.add_trace(go.Scatter(
            x=[df_ticker['Date'].min(), df_ticker['Date'].max()],
            y=[y_val, y_val],
            mode='lines',
            name=level,
            line=dict(dash='dash')
        ))

    fig.update_layout(
        title=f'{ticker} - Last Day Pivot Levels',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template='plotly_dark'
    )

    fig.show()


In [17]:
plot_pivots(df,'ABB')

No data for ticker: ABB


In [18]:
df.head()

Date      Ticker       Close        High         Low  \
Ticker                                                                     
ADANIGREEN 0  2020-06-30  ADANIGREEN  358.850006  358.850006  358.850006   
           10 2020-07-01  ADANIGREEN  340.950012  340.950012  340.950012   
           20 2020-07-02  ADANIGREEN  357.950012  357.950012  323.950012   
           30 2020-07-03  ADANIGREEN  375.799988  375.799988  375.799988   
           40 2020-07-06  ADANIGREEN  394.549988  394.549988  362.250000   

                     Open     Volume  SMA_20  SMA_50  SMA_100  ...  \
Ticker                                                         ...   
ADANIGREEN 0   358.850006   126347.0     NaN     NaN      NaN  ...   
           10  340.950012    80912.0     NaN     NaN      NaN  ...   
           20  323.950012  3295000.0     NaN     NaN      NaN  ...   
           30  375.799988   127962.0     NaN     NaN      NaN  ...   
           40  394.549988  5078968.0     NaN     NaN      NaN  ...   

                  EMA_200  upper_band_bb  lower_band_bb       Pivot  \
Ticker                                                                
ADANIGREEN 0   358.850006            NaN            NaN  358.850006   
           10  358.671897            NaN            NaN  340.950012   
           20  358.664714            NaN            NaN  346.616679   
           30  358.835214            NaN            NaN  375.799988   
           40  359.190585            NaN            NaN  383.783325   

                       S1          S2          S3          R1          R2  \
Ticker                                                                      
ADANIGREEN 0   358.850006  358.850006  358.850006  358.850006  358.850006   
           10  340.950012  340.950012  340.950012  340.950012  340.950012   
           20  335.283346  312.616679  278.616679  369.283346  380.616679   
           30  375.799988  375.799988  375.799988  375.799988  375.799988   
           40  373.016663  351.483337  319.183350  405.316650  416.083313   

                       R3  
Ticker                     
ADANIGREEN 0   358.850006  
           10  340.950012  
           20  414.616679  
           30  375.799988  
           40  448.383301  

[5 rows x 23 columns]

In [19]:
import pandas as pd

def ichimoku(df, stock_col):
    # 🔍 Debug: Print current index and column names
    print("Index names before fix:", df.index.names)
    print("Column names before fix:", df.columns.tolist())

    # ✅ Fix: Ensure 'stock_col' is not in index
    if stock_col in df.index.names:
        df = df.reset_index(drop=True)
    elif stock_col in df.index.names:
        df = df.reset_index()

    # ✅ Remove duplicate columns if any
    df = df.loc[:, ~df.columns.duplicated()]

    # 🔍 Debug after fix
    print("Index names after fix:", df.index.names)
    print("Column names after fix:", df.columns.tolist())

    # 🎯 Define how to compute Ichimoku indicators for each group
    def compute_indicators(group):
        group = group.sort_values('Date')
        group['Tenkan_Sen'] = (group['High'].rolling(window=9).max() + group['Low'].rolling(window=9).min()) / 2
        group['Kijun_Sen'] = (group['High'].rolling(window=26).max() + group['Low'].rolling(window=26).min()) / 2
        group['Senkou_Span_A'] = ((group['Tenkan_Sen'] + group['Kijun_Sen']) / 2).shift(26)
        group['Senkou_Span_B'] = ((group['High'].rolling(window=52).max() + group['Low'].rolling(window=52).min()) / 2).shift(26)
        group['Chikou_Span'] = group['Close'].shift(-26)
        return group

    # ✅ Apply safely
    df = df.groupby(stock_col, group_keys=False).apply(compute_indicators)

    # Reset index in case groupby changed it
    df = df.reset_index(drop=True)

    return df


In [20]:
df = ichimoku(df, 'Ticker')

Index names before fix: ['Ticker', None]
Column names before fix: ['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_20', 'SMA_50', 'SMA_100', 'SMA_200', 'EMA_50', 'EMA_100', 'EMA_200', 'upper_band_bb', 'lower_band_bb', 'Pivot', 'S1', 'S2', 'S3', 'R1', 'R2', 'R3']
Index names after fix: [None]
Column names after fix: ['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_20', 'SMA_50', 'SMA_100', 'SMA_200', 'EMA_50', 'EMA_100', 'EMA_200', 'upper_band_bb', 'lower_band_bb', 'Pivot', 'S1', 'S2', 'S3', 'R1', 'R2', 'R3']


C:\Users\katch\AppData\Local\Temp\ipykernel_24192\3161390799.py:32: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [21]:
df.columns

Index(['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_20',
       'SMA_50', 'SMA_100', 'SMA_200', 'EMA_50', 'EMA_100', 'EMA_200',
       'upper_band_bb', 'lower_band_bb', 'Pivot', 'S1', 'S2', 'S3', 'R1', 'R2',
       'R3', 'Tenkan_Sen', 'Kijun_Sen', 'Senkou_Span_A', 'Senkou_Span_B',
       'Chikou_Span'],
      dtype='object')

In [22]:
def plot_ichimoku_cloud(df,ticker):
    plot(df,ticker,['Senkou_Span_A','Senkou_Span_B'])

In [23]:
plot_ichimoku_cloud(df,'ABB')

In [24]:
def MACD_Indicator(df,stock_col):
    df.groupby(stock_col).apply(lambda x: x.sort_values('Date',inplace=True))
    df['MACD']=df['Close'].ewm(span=12,adjust=False).mean()-df['Close'].ewm(span=26,adjust=False).mean()
    df['Signal']=df['MACD'].ewm(span=9,adjust=False).mean()
    df['MACD_Histogram']=df['MACD']-df['Signal']
    return df




In [25]:
df=MACD_Indicator(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\2284535264.py:2: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [26]:
plot(df,'ABB',['MACD','Signal'])

In [27]:
df[df['Ticker']=='ABB'].tail()

,Date,Ticker,Close,High,Low,Open,Volume,SMA_20,SMA_50,SMA_100,...,R2,R3,Tenkan_Sen,Kijun_Sen,Senkou_Span_A,Senkou_Span_B,Chikou_Span,MACD,Signal,MACD_Histogram


In [28]:
def RSI_Indicator(df, stock_col, period=14):
    def compute_rsi(group):
        group = group.sort_values('Date')
        delta = group['Close'].diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)

        avg_gain = gain.rolling(window=period, min_periods=period).mean()
        avg_loss = loss.rolling(window=period, min_periods=period).mean()

        rs = avg_gain / avg_loss
        group['RSI'] = 100 - (100 / (1 + rs))
        return group

    return df.groupby(stock_col, group_keys=False).apply(compute_rsi)


In [29]:
df=RSI_Indicator(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\855197247.py:15: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [30]:
import plotly.graph_objects as go

def plot_rsi(df, ticker):
    df_ticker = df[df['Ticker'] == ticker].copy()
    df_ticker = df_ticker.sort_values('Date')
    df_ticker = df_ticker.dropna(subset=['RSI'])

    fig = go.Figure()

    # RSI line
    fig.add_trace(go.Scatter(
        x=df_ticker['Date'],
        y=df_ticker['RSI'],
        mode='lines',
        name='RSI',
        line=dict(color='purple')
    ))

    # Overbought line
    fig.add_hline(
        y=70, 
        line=dict(color='red', dash='dash'),
        annotation_text='Overbought',
        annotation_position='top left',
        showlegend=False
    )

    # Neutral line (50)
    fig.add_hline(
        y=50, 
        line=dict(color='gray', dash='dash'),
        annotation_text='Neutral',
        annotation_position='top right',
        showlegend=False
    )

    # Oversold line
    fig.add_hline(
        y=30, 
        line=dict(color='green', dash='dash'),
        annotation_text='Oversold',
        annotation_position='bottom left',
        showlegend=False
    )

    # Layout
    fig.update_layout(
        title=f'{ticker} RSI Indicator',
        xaxis_title='Date',
        yaxis_title='RSI',
        yaxis=dict(range=[0, 100]),
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()


In [31]:
plot_rsi(df,'ABB')

In [32]:
def compute_atr(df, stock_col, period=14):
    def calculate(group):
        group = group.sort_values('Date')
        high_low = group['High'] - group['Low']
        high_close = (group['High'] - group['Close'].shift()).abs()
        low_close = (group['Low'] - group['Close'].shift()).abs()

        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        group['ATR'] = tr.rolling(window=period).mean()
        return group

    return df.groupby(stock_col, group_keys=False).apply(calculate)


In [33]:
df=compute_atr(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\3314405725.py:12: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [34]:
import plotly.graph_objects as go

def plot_atr(df, ticker):
    df_ticker = df[df['Ticker'] == ticker].copy()
    df_ticker = df_ticker.sort_values('Date')
    df_ticker = df_ticker.dropna(subset=['ATR'])

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_ticker['Date'],
        y=df_ticker['ATR'],
        mode='lines',
        name='ATR',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title=f'{ticker} ATR (Average True Range)',
        xaxis_title='Date',
        yaxis_title='ATR',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()


In [35]:
plot_atr(df,'ABB')

In [36]:
def stochastic_oscillator(df,stock_col):
    def calculate(group):
        group = group.sort_values('Date')
        l14=df['Low'].rolling(window=14).min()
        h14=df['High'].rolling(window=14).max()
        c=df['Close']
        k=((c-l14)/(h14-l14))*100
        d=k.rolling(window=3).mean()
        group['so_K']=k
        group['so_D']=d
        return group
    return df.groupby(stock_col,group_keys=False).apply(calculate)

In [37]:
df=stochastic_oscillator(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\1911064787.py:12: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [38]:
import plotly.graph_objects as go
def plot_stochastic_oscillator(df,ticker):
    df_ticker=df[df['Ticker']==ticker].copy()
    df_ticker=df_ticker.sort_values('Date')
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=df_ticker['Date'],y=df_ticker['so_K'],mode='lines',name='K'))
    fig.add_trace(go.Scatter(x=df_ticker['Date'],y=df_ticker['so_D'],mode='lines',name='D'))
    fig.update_layout(title=f'{ticker} Stochastic Oscillator',xaxis_title='Date',yaxis_title='Value',template='plotly_dark',hovermode='x unified')
    fig.show()
    

In [39]:
plot_stochastic_oscillator(df,'ABB')

In [40]:
def sar_indicator(df, stock_col, af_step=0.02, af_max=0.2):
    def calculate(group):
        group = group.sort_values('Date').reset_index(drop=True)

        sar = []
        trend = []
        ep = group['High'].iloc[0]
        af = af_step
        sar_val = group['Low'].iloc[0]
        uptrend = True

        for i in range(len(group)):
            if i == 0:
                sar.append(sar_val)
                trend.append(uptrend)
                continue

            prev_sar = sar[-1]

            if uptrend:
                new_sar = prev_sar + af * (ep - prev_sar)
                new_sar = min(new_sar, group['Low'].iloc[i - 1], group['Low'].iloc[i])
                if group['Low'].iloc[i] < new_sar:
                    uptrend = False
                    sar_val = ep
                    ep = group['Low'].iloc[i]
                    af = af_step
                else:
                    sar_val = new_sar
                    if group['High'].iloc[i] > ep:
                        ep = group['High'].iloc[i]
                        af = min(af + af_step, af_max)
            else:
                new_sar = prev_sar + af * (ep - prev_sar)
                new_sar = max(new_sar, group['High'].iloc[i - 1], group['High'].iloc[i])
                if group['High'].iloc[i] > new_sar:
                    uptrend = True
                    sar_val = ep
                    ep = group['High'].iloc[i]
                    af = af_step
                else:
                    sar_val = new_sar
                    if group['Low'].iloc[i] < ep:
                        ep = group['Low'].iloc[i]
                        af = min(af + af_step, af_max)

            sar.append(sar_val)
            trend.append(uptrend)

        group['SAR'] = sar
        group['Trend_Up'] = trend
        return group

    return df.groupby(stock_col, group_keys=False).apply(calculate)


In [41]:
df=sar_indicator(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\2911544851.py:54: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [42]:
def plot_sar(df, ticker):
    df_ticker = df[df['Ticker'] == ticker].copy()
    df_ticker = df_ticker.sort_values('Date')

    fig = go.Figure()

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=df_ticker['Date'],
        open=df_ticker['Open'],
        high=df_ticker['High'],
        low=df_ticker['Low'],
        close=df_ticker['Close'],
        name='Candlestick'
    ))

    # SAR
    fig.add_trace(go.Scatter(
        x=df_ticker['Date'],
        y=df_ticker['SAR'],
        mode='markers',
        name='SAR',
        marker=dict(color='cyan', size=5, symbol='circle')
    ))

    fig.update_layout(
        title=f'{ticker} Parabolic SAR',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()


In [43]:
plot_sar(df,'ABB')

In [44]:
def fibonacci_retracement(df, stock_col):
    def calculate(group):
        group = group.sort_values('Date')
        high = group['High'].max()
        low = group['Low'].min()
        diff = high - low

        fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
        for level in fib_levels:
            group[f'Fib_{float(level*100)}'] = high - diff * level

        return group

    return df.groupby(stock_col, group_keys=False).apply(calculate)


In [45]:
df=fibonacci_retracement(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_24192\2729463791.py:14: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [46]:
def plot_fibonacci(df, ticker):
    df_ticker = df[df['Ticker'] == ticker].copy()
    df_ticker = df_ticker.sort_values('Date')

    fib_cols = [col for col in df_ticker.columns if col.startswith('Fib_')]

    fig = go.Figure()

    # Add candlestick
    fig.add_trace(go.Candlestick(
        x=df_ticker['Date'],
        open=df_ticker['Open'],
        high=df_ticker['High'],
        low=df_ticker['Low'],
        close=df_ticker['Close'],
        name='Candlestick'
    ))

    # Add fib levels (use last row for horizontal lines)
    last = df_ticker.iloc[-1]
    for col in fib_cols:
        fig.add_hline(y=last[col], line_dash='dash', annotation_text=col, annotation_position='right')

    fig.update_layout(
        title=f'{ticker} Fibonacci Retracement',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_dark'
    )

    fig.show()


In [47]:
plot_fibonacci(df,'ABB')

IndexError: single positional indexer is out-of-bounds

In [48]:
def obv_indicator(df, stock_col):
    def calculate(group):
        group = group.sort_values('Date').copy()
        obv = [0]

        for i in range(1, len(group)):
            if group['Close'].iloc[i] > group['Close'].iloc[i - 1]:
                obv.append(obv[-1] + group['Volume'].iloc[i])
            elif group['Close'].iloc[i] < group['Close'].iloc[i - 1]:
                obv.append(obv[-1] - group['Volume'].iloc[i])
            else:
                obv.append(obv[-1])

        group['OBV'] = obv
        return group

    return df.groupby(stock_col, group_keys=False).apply(calculate)


In [49]:
df=obv_indicator(df,'Ticker')

C:\Users\katch\AppData\Local\Temp\ipykernel_14724\1190672643.py:17: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [50]:
def plot_obv(df, ticker):
    df_ticker = df[df['Ticker'] == ticker].copy()
    df_ticker = df_ticker.sort_values('Date')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_ticker['Date'], y=df_ticker['OBV'], mode='lines', name='OBV', line=dict(color='cyan')))
    
    fig.update_layout(
        title=f'{ticker} On-Balance Volume (OBV)',
        xaxis_title='Date',
        yaxis_title='OBV',
        template='plotly_dark'
    )

    fig.show()


In [51]:
plot_obv(df,'ABB')

In [55]:
l=df.columns.tolist()
len(l)

45

In [57]:
print(l)

['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_20', 'SMA_50', 'SMA_100', 'SMA_200', 'EMA_50', 'EMA_100', 'EMA_200', 'upper_band_bb', 'lower_band_bb', 'Pivot', 'S1', 'S2', 'S3', 'R1', 'R2', 'R3', 'Tenkan_Sen', 'Kijun_Sen', 'Senkou_Span_A', 'Senkou_Span_B', 'Chikou_Span', 'MACD', 'Signal', 'MACD_Histogram', 'RSI', 'ATR', 'so_K', 'so_D', 'SAR', 'Trend_Up', 'Fib_0.0', 'Fib_23.599999999999998', 'Fib_38.2', 'Fib_50.0', 'Fib_61.8', 'Fib_78.60000000000001', 'Fib_100.0', 'OBV']


In [61]:
d=df.head(1).to_dict()
print(d)

{'Date': {0: Timestamp('2020-06-22 00:00:00')}, 'Ticker': {0: 'ABB'}, 'Close': {0: 811.4605102539062}, 'High': {0: 818.772722095988}, 'Low': {0: 801.4671540697277}, 'Open': {0: 804.3433026289997}, 'Volume': {0: 175107.0}, 'SMA_20': {0: nan}, 'SMA_50': {0: nan}, 'SMA_100': {0: nan}, 'SMA_200': {0: nan}, 'EMA_50': {0: 811.4605102539062}, 'EMA_100': {0: 811.4605102539062}, 'EMA_200': {0: 811.4605102539062}, 'upper_band_bb': {0: nan}, 'lower_band_bb': {0: nan}, 'Pivot': {0: 810.5667954732074}, 'S1': {0: 802.3608688504269}, 'S2': {0: 793.2612274469471}, 'S3': {0: 775.9556594206869}, 'R1': {0: 819.6664368766872}, 'R2': {0: 827.8723634994677}, 'R3': {0: 845.177931525728}, 'Tenkan_Sen': {0: nan}, 'Kijun_Sen': {0: nan}, 'Senkou_Span_A': {0: nan}, 'Senkou_Span_B': {0: nan}, 'Chikou_Span': {0: 876.7492065429688}, 'MACD': {0: 0.0}, 'Signal': {0: 0.0}, 'MACD_Histogram': {0: 0.0}, 'RSI': {0: nan}, 'ATR': {0: nan}, 'so_K': {0: nan}, 'so_D': {0: nan}, 'SAR': {0: 801.4671540697277}, 'Trend_Up': {0: Tru